<a href="https://colab.research.google.com/github/juanfranbrv/crewAI/blob/main/crewai_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuración del entorno del cuaderno**

In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
# GROQ_API_KEY=userdata.get('GROQ_API_KEY')
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
# %pip install langchain_community -qU  # Instalar la librería principal de LangChain.
# %pip install crewai -qU
# %pip install crewai_tools -qU

%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

from crewai import Agent, Task, Crew

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
# %pip install langchain-groq -qU
# %pip install langchain-google-genai -qU
# %pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# from langchain.prompts import ChatPromptTemplate


# Importamos las clases necesarias para trabajar con cadenas
# from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

**Role** = Define la función y la experiencia del agente dentro del equipo.  
**Goal** = El objetivo individual que guía la toma de decisiones del agente.  
**Backstory** = Aporta contexto y personalidad al agente, enriqueciendo las interacciones.


In [1]:
%%capture --no-stderr

%pip install crewai
%pip install crewai_tools
%pip install langchain_community -qU
# %pip install langchain-openai -qU
# %pip install pydantic

In [1]:
%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [2]:
from crewai import Agent, Task, Crew
# from langchain_openai import ChatOpenAI

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [3]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

Enter your OpenAI API key: ··········


In [4]:
planificador = Agent(
    role="Planificador de contenidos",
    goal="Planifique contenido atractivo y objetivamente preciso en {topic}",
    backstory="Estás trabajando en la planificación de un artículo de blog "
              "sobre el tema: {topic}"
              "Recopilas información que ayuda al "
              "la audiencia a aprender algo"
              "y tomar decisiones informadas"
              "Tu trabajo es la base para que"
              "el redactor de contenido pueda escribir un artículo sobre este tema.",
    allow_delegation=False,
	verbose=True,
    LLM="gpt-4o-mini"
)

escritor = Agent(
    role="Escritor",
    goal="Escribir un artículo de opinión perspicaz y veraz sobre el tema: {topic}",
    backstory="Estás trabajando en la redacción de un nuevo artículo de opinión sobre el tema: {topic}"
                "Basas tu redacción en el trabajo del "
                "Planificador de contenido, que proporciona un esquema "
                "y un contexto relevante sobre el tema"
                "Sigues los objetivos principales y la "
                "dirección del esquema, "
                "tal como lo proporciona el Planificador de contenido"
                "También proporcionas perspectivas objetivas e imparciales "
                "y las respaldas con la información "
                "proporcionada por el Planificador de contenido"
                "Reconoces en tu artículo de opinión "
                "cuando tus declaraciones son opiniones "
                "en lugar de declaraciones objetivas",
    allow_delegation=False,
    verbose=True,
    LLM="gpt-4o-mini"
)

editor = Agent(
    role="Editor",
    goal="Editar una publicación de blog determinada para alinearla con "
            "el estilo de escritura de la organización",
    backstory="Eres un editor que recibe una publicación de blog"
                "del escritor"
                "Tu objetivo es revisar la publicación de blog"
                "para asegurarse de que sigue las mejores prácticas periodísticas"
                "ofrece puntos de vista equilibrados"
                "al brindar opiniones o afirmaciones"
                "y también evita temas controvertidos importantes"
                "u opiniones cuando sea posible",
    allow_delegation=False,
    verbose=True,
    LLM="gpt-4o-mini"
)

In [5]:
planear = Task(
    description=(
        "1. Priorizar las últimas tendencias, los actores clave y las noticias destacadas sobre {topic}.\n"
        "2. Identificar la audiencia objetivo, teniendo en cuenta sus intereses y sus puntos débiles.\n"
        "3. Desarrollar un esquema de contenido detallado que incluya una introducción, puntos clave y un llamado a la acción.\n"
        "4. Incluir palabras clave de SEO y datos o fuentes relevantes"
    ),
    expected_output="Un documento de plan de contenido completo"
                    "con un esquema, análisis de audiencia"
                    "palabras clave de SEO y recursos",
    agent=planificador,
)

escribir = Task(
    description=(
        "1. Utilice el plan de contenido para elaborar una publicación de blog atractiva "
            "sobre {topic}.\n"
            "2. Incorpore palabras clave de SEO de forma natural.\n"
            "3. Las secciones y los subtítulos tienen el nombre correcto "
            "de manera atractiva.\n"
            "4. Asegúrese de que la publicación esté estructurada con una "
            "introducción atractiva, un cuerpo perspicaz "
            "y una conclusión resumida.\n"
            "5. Revise los errores gramaticales y "
            "que se alinee con la voz de la marca.\n"
    ),
    expected_output="Una entrada de blog bien escrita"
                    "en formato Markdown, lista para publicar"
                    "cada sección debe tener 2 o 3 párrafos",
    agent=escritor,
)

editar = Task(
    description=("Revise la publicación del blog para detectar "
                    "errores gramaticales y "
                    "alineación con la voz de la marca"),
    expected_output="Una entrada de blog bien escrita"
                    "en formato Markdown, lista para publicar"
                    "cada sección debe tener 2 o 3 párrafos",
    agent=editor
)

In [6]:
crew = Crew(
    agents=[planificador, escritor, editor],
    tasks=[planear, escribir, editar],
    verbose=2
)

In [7]:
result = crew.kickoff(inputs={"topic": "Streamlit"})

 [DEBUG]: == Working Agent: Planificador de contenidos
 [INFO]: == Starting Task: 1. Priorizar las últimas tendencias, los actores clave y las noticias destacadas sobre Streamlit.
2. Identificar la audiencia objetivo, teniendo en cuenta sus intereses y sus puntos débiles.
3. Desarrollar un esquema de contenido detallado que incluya una introducción, puntos clave y un llamado a la acción.
4. Incluir palabras clave de SEO y datos o fuentes relevantes


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

1. Últimas tendencias, actores clave y noticias destacadas sobre Streamlit:
Streamlit se ha convertido en una herramienta popular entre los científicos de datos y los desarrolladores web. Recientemente, Streamlit lanzó su versión 0.84.0 que introdujo nuevas características como la personalización de temas. Los actores clave en la comunidad de Streamlit incluyen a su cofundador, Adrien Treuille, y a otros contribuyentes destacados como Randy Zwitch. Una 

In [8]:
from IPython.display import Markdown
Markdown(result)

# Streamlit: La Herramienta de Desarrollo de Aplicaciones de Datos Que está Revolucionando el Campo

Como científico de datos, desarrollador web o entusiasta de la codificación, hay una plataforma que debes tener en tu radar: Streamlit. Esta herramienta, que ha ido adquiriendo popularidad, está transformando la manera en que desarrollamos aplicaciones web de datos. Gracias a Streamlit, se pueden convertir scripts de Python en aplicaciones web interactivas de manera rápida y sencilla, facilitando una experimentación más eficiente y una implementación más suave.

## Streamlit: Avances Recientes y Figuras Destacadas

La versión más reciente de Streamlit, la 0.84.0, ha incorporado una serie de características novedosas, incluyendo la personalización de temas. Esto brinda a los usuarios la posibilidad de ajustar el aspecto de sus aplicaciones para que reflejen mejor sus gustos o la imagen de su marca.

En la comunidad de Streamlit, hay varias figuras sobresalientes, entre las que destacan algunas. Adrien Treuille, cofundador de Streamlit, ha sido una presencia importante en su desarrollo. Otros colaboradores destacados son personas como Randy Zwitch, que ha contribuido enormemente a la comunidad.

Una de las noticias más relevantes de Streamlit es su reciente ronda de financiación de serie B, en la que recaudó $35 millones. Este capital permitirá a Streamlit ampliar su plataforma y seguir enriqueciendo sus funcionalidades. Esto representa un paso significativo para Streamlit y promete grandes avances en el futuro.

## Únete a la Comunidad de Streamlit

Si buscas simplificar y agilizar el desarrollo de aplicaciones web de datos, te animamos a que pruebes Streamlit. La plataforma es fácil de utilizar y cuenta con una comunidad activa y receptiva de científicos de datos y desarrolladores web.

Para profundizar en Streamlit, te sugerimos visitar el [blog oficial de Streamlit](https://blog.streamlit.io), donde encontrarás tutoriales valiosos y noticias sobre las últimas actualizaciones. También puedes consultar la [documentación de Streamlit](https://docs.streamlit.io) para obtener una guía detallada sobre cómo sacarle el máximo provecho a la plataforma. Y, por supuesto, no olvides participar en los [foros de la comunidad de Streamlit](https://discuss.streamlit.io) para obtener consejos, trucos y asistencia de otros usuarios de Streamlit.

Streamlit está cambiando las reglas del juego en el desarrollo de aplicaciones de datos. No te quedes atrás, ¡forma parte de la revolución Streamlit hoy mismo!